# Validation Definitions and Checkpoints

Before you check this out, make sure you're gone through "Authoring Expectation Suites", which connects to data.


### Imports

In [ ]:
import great_expectations as gx
import great_expectations.exceptions as exceptions
from great_expectations.checkpoint.v1_checkpoint import Checkpoint
from great_expectations.core.validation_definition import ValidationDefinition
from great_expectations.datasource.fluent.interfaces import Datasource

from constants import (
    CHECKPOINT_NAME,
    DATASOURCE_NAME,
    ASSET_NAME,
    SUITE_NAME,
    BATCH_DEFINITION_NAME_WHOLE_TABLE,
)

### Load our entities

In [ ]:
context = gx.get_context(mode="file")

datasource = context.get_datasource(DATASOURCE_NAME)
assert isinstance(datasource, Datasource)
data_asset = datasource.get_asset(asset_name=ASSET_NAME)

suite = context.suites.get(SUITE_NAME)
batch_definition = next(bd for bd in data_asset.batch_definitions if bd.name == BATCH_DEFINITION_NAME_WHOLE_TABLE)

### Create a Validation Definition

Validation definitions tie your data to an expectation Suite

In [ ]:
validation_definition = ValidationDefinition(
    name="ephemeral validation definition",
    data=batch_definition,
    suite=suite,
)

results = validation_definition.run()
print(results)

## Create and run a Checkpoint

In [ ]:
try:
    checkpoint = context.checkpoints.add(Checkpoint(
        name=CHECKPOINT_NAME,
        validation_definitions=[
            validation_definition,
            ValidationDefinition(name="another", data=batch_definition, suite=suite)
        ],
    ))
except exceptions.DataContextError:
    checkpoint = context.checkpoints.get(name=CHECKPOINT_NAME)

results = checkpoint.run()
print(results)